In [1]:
from om_ogc_timeseries_client import OmOgcTimeseriesClient

client = OmOgcTimeseriesClient({"token":"whos-36e7d1a3-b61e-4ae0-aca4-60efe6f27361"})

site = "F426CC5C7D4A1563EE3B41E4F50FB4DE8568D3F2"
variable = "B9240615C51DA438E9E02C9858887F1EE726EE2E"
beginDate = "1990-01-01"
endDate = "2024-05-01"

data = client.getData(site, variable, beginDate, endDate)


In [ ]:
data

In [ ]:
import pandas
df = pandas.DataFrame(data)
df["date"] = pandas.to_datetime(df["date"])
df = df.set_index("date")
df